In [32]:
# for reading data
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

# for modeling
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

import plotly as plt



In [ ]:
example website
https://medium.com/luca-chuangs-bapm-notes/build-a-neural-network-in-python-multi-class-classification-e940f74bd899

In [3]:
#read in the data
#take input from provisional database (csv)
# Load the csv file from GitHub
url = 'https://raw.githubusercontent.com/mandymccabe/Final_Project/janet_branch/Data/Final_Project_Full.csv'
url2= 'https://raw.githubusercontent.com/mandymccabe/Final_Project/main/Resources/all_responses_coded.csv'
df = pd.read_csv(url, index_col=0)
df2 = pd.read_csv(url2, index_col=0)

In [11]:
clean_df2= df2.drop(['A1','A2','A3','A4','A5','A21','StartDate','EndDate'], axis=1)

In [9]:
PoliticalViews = df.filter(["political_views"], axis=1)
PoliticalViews.head()

,political_views
respondentid,
6176264298,Moderate
6176263960,Moderate
6176258621,Liberal
6176257082,Liberal
6176256111,Liberal


In [12]:
merged_dfs = pd.merge(clean_df2, PoliticalViews, how='outer', left_on=["RespondentID"], right_on=['respondentid'])
merged_dfs.head()

,RespondentID,A6,A7,A8,A9,A10,A11,A12,A13,A14,...,A54,A55,A56,A57,A58,A59,A60,A61,A62,political_views
0,6176264298,0,1,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,Moderate
1,6176263960,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Moderate
2,6176258621,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,Liberal
3,6176257082,0,1,0,0,0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,Liberal
4,6176256111,0,1,0,0,1,1,1,0,0,...,0,0,1,1,0,0,0,0,0,Liberal


In [14]:
# split into X and Y
Y = merged_dfs['political_views']
X = merged_dfs.drop(['political_views'], axis=1)

print(X.shape)
print(Y.shape)

# convert to numpy arrays
X = np.array(X)

(1021, 57)
(1021,)


In [15]:
# show Y
Y.head()

0    Moderate
1    Moderate
2     Liberal
3     Liberal
4     Liberal
Name: political_views, dtype: object

In [16]:
# work with labels
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


In [17]:
print(encoded_Y)

[2 2 1 ... 2 2 2]


In [18]:
print(dummy_y)

[[0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]


In [26]:
# build a model
model = Sequential()
model.add(Dense(16, input_shape=(X.shape[1],), activation='relu')) # input shape is (features,)
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 16)                928       
                                                                 
 dense_3 (Dense)             (None, 5)                 85        
                                                                 
Total params: 1,013
Trainable params: 1,013
Non-trainable params: 0
_________________________________________________________________


In [27]:
# compile the model
model.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy', # this is different instead of binary_crossentropy (for regular classification)
              metrics=['accuracy'])

In [28]:
import keras
from keras.callbacks import EarlyStopping

# early stopping callback
# This callback will stop the training when there is no improvement in  
# the validation loss for 10 consecutive epochs.  
es = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                   mode='min',
                                   patience=10, 
                                   restore_best_weights=True) # important - otherwise you just return the last weigths...



In [30]:
# now we just update our model fit call
history = model.fit(X,
                    dummy_y,
                    callbacks=[es],
                    epochs=8000000, # you can set this to a big number!
                    batch_size=10,
                    shuffle=True,
                    validation_split=0.2,
                    verbose=1
                    )

Epoch 1/8000000
82/82 [==============================] - 0s 4ms/step - loss: 14428249.0000 - accuracy: 0.2463 - val_loss: 6864615.0000 - val_accuracy: 0.1902
Epoch 2/8000000
82/82 [==============================] - 0s 2ms/step - loss: 3680687.0000 - accuracy: 0.2684 - val_loss: 655024.2500 - val_accuracy: 0.4293
Epoch 3/8000000
82/82 [==============================] - 0s 2ms/step - loss: 980481.8750 - accuracy: 0.2574 - val_loss: 494288.3125 - val_accuracy: 0.4293
Epoch 4/8000000
82/82 [==============================] - 0s 2ms/step - loss: 994571.7500 - accuracy: 0.2708 - val_loss: 1155320.1250 - val_accuracy: 0.1902
Epoch 5/8000000
82/82 [==============================] - 0s 2ms/step - loss: 941371.6250 - accuracy: 0.2819 - val_loss: 401958.0938 - val_accuracy: 0.1902
Epoch 6/8000000
82/82 [==============================] - 0s 3ms/step - loss: 972451.1875 - accuracy: 0.2623 - val_loss: 834067.1875 - val_accuracy: 0.4293
Epoch 7/8000000
82/82 [==============================] - 0s 2ms/s

In [35]:

history_dict = history.history

# learning curve
# accuracy
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']


In [36]:
acc

[0.24632352590560913,
 0.26838234066963196,
 0.25735294818878174,
 0.2708333432674408,
 0.281862735748291,
 0.26225489377975464,
 0.2696078419685364,
 0.2904411852359772,
 0.2757352888584137,
 0.28799018263816833,
 0.24632352590560913,
 0.25735294818878174,
 0.25245097279548645,
 0.26348039507865906,
 0.26225489377975464,
 0.24754901230335236,
 0.2696078419685364,
 0.27328431606292725,
 0.2561274468898773,
 0.25857841968536377]

In [37]:
val_acc

[0.1902438998222351,
 0.4292683005332947,
 0.4292683005332947,
 0.1902438998222351,
 0.1902438998222351,
 0.4292683005332947,
 0.1902438998222351,
 0.4292683005332947,
 0.20975609123706818,
 0.20975609123706818,
 0.1902438998222351,
 0.10731707513332367,
 0.1902438998222351,
 0.1902438998222351,
 0.20975609123706818,
 0.10731707513332367,
 0.4292683005332947,
 0.06341463327407837,
 0.4292683005332947,
 0.4292683005332947]

In [38]:
# loss
loss = history_dict['loss']
val_loss = history_dict['val_loss']


In [39]:
loss

[14428249.0,
 3680687.0,
 980481.875,
 994571.75,
 941371.625,
 972451.1875,
 892690.1875,
 909846.6875,
 869063.0,
 892047.9375,
 868041.125,
 827997.25,
 824665.3125,
 794480.1875,
 784058.5625,
 792034.4375,
 786895.5,
 759607.9375,
 757663.0,
 728055.125]

In [40]:
val_loss

[6864615.0,
 655024.25,
 494288.3125,
 1155320.125,
 401958.09375,
 834067.1875,
 1678461.5,
 468851.75,
 1441701.625,
 260189.625,
 1503543.75,
 1918658.375,
 1036081.6875,
 1367514.125,
 307728.8125,
 385074.96875,
 941168.25,
 835881.1875,
 940327.0,
 567675.25]

In [43]:

# range of X (no. of epochs)
epochs = range(1, len(acc) + 1)



In [45]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

preds = model.predict(X) # see how the model did!



32/32 [==============================] - 0s 4ms/step


In [46]:
print(preds[0]) # i'm spreading that prediction across three nodes and they sum to 1

[0. 1. 0. 0. 0.]


In [47]:
print(np.sum(preds[0]))

1.0


In [48]:
# Almost a perfect prediction
# actual is left, predicted is top
# names can be found by inspecting Y
matrix = confusion_matrix(dummy_y.argmax(axis=1), preds.argmax(axis=1))
matrix

array([[  0, 215,   0,   0,   0],
       [  0, 220,   0,   0,   0],
       [  0, 420,   0,   0,   0],
       [  0,  55,   0,   0,   0],
       [  0, 111,   0,   0,   0]], dtype=int64)

In [49]:
print(classification_report(dummy_y.argmax(axis=1), preds.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       215
           1       0.22      1.00      0.35       220
           2       0.00      0.00      0.00       420
           3       0.00      0.00      0.00        55
           4       0.00      0.00      0.00       111

    accuracy                           0.22      1021
   macro avg       0.04      0.20      0.07      1021
weighted avg       0.05      0.22      0.08      1021



c:\Users\m099875\Anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\m099875\Anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\m099875\Anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
